In [1]:
import sys
sys.path.append('..')
import os
import sklearn.metrics.cluster
import sklearn.cluster
import seaborn as sns
import scanpy as sc
import sklearn as sk
from sklearn.neighbors import NearestNeighbors
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import time
import numpy as np
import pandas as pd
import pickle
import utils
import anndata
import gc
import benchmark_common_functions
import csv
from itertools import product
import pingouin as pg

/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.7, the latest is 0.3.10.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
base_folder = '../data/scRNAseq_Benchmark_datasets/Intra-dataset/'
base_folder2 = '../data/scRNAseq_Benchmark_datasets/Inter-dataset/'
resdir = '../results/scRNAseq_Benchmark_datasets/'

get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_sc = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}_scanpyhubness.pkl'

all_data_paths = dict(
    #AMB
    AMB3={'data':base_folder+'AMB/Filtered_mouse_allen_brain_data.csv',
         'labels':base_folder+'AMB/Labels.csv',
         'labels_key':'Class'},
    
    AMB16={'data':base_folder+'AMB/Filtered_mouse_allen_brain_data.csv',
         'labels':base_folder+'AMB/Labels.csv',
         'labels_key':'Subclass'},
    
    AMB92={'data':base_folder+'AMB/Filtered_mouse_allen_brain_data.csv',
         'labels':base_folder+'AMB/Labels.csv',
         'labels_key':'cluster'},
    
    #CellBench
    cellbench1={'data':base_folder+'CellBench/10x_5cl/10x_5cl_data.csv',
                'labels':base_folder+'CellBench/10x_5cl/Labels.csv',
                'labels_key':'x'},
    
    cellbench2={'data':base_folder+'CellBench/CelSeq2_5cl/CelSeq2_5cl_data.csv',
                'labels':base_folder+'CellBench/CelSeq2_5cl/Labels.csv',
                'labels_key':'x'},

    #Pancreatic
    BaronHuman={'data':base_folder+'Pancreatic_data/Baron Human/Filtered_Baron_HumanPancreas_data.csv',
            'labels':base_folder+'Pancreatic_data/Baron Human/Labels.csv',
            'labels_key':'x'},
    BaronMouse={'data':base_folder+'Pancreatic_data/Baron Mouse/Filtered_MousePancreas_data.csv',
                'labels':base_folder+'Pancreatic_data/Baron Mouse/Labels.csv',
                'labels_key':'x'},
    Muraro={'data':base_folder+'Pancreatic_data/Muraro/Filtered_Muraro_HumanPancreas_data.csv',
                'labels':base_folder+'Pancreatic_data/Muraro/Labels.csv',
           'labels_key':'x'},
    Segerstolpe={'data':base_folder+'Pancreatic_data/Segerstolpe/Filtered_Segerstolpe_HumanPancreas_data.csv',
                'labels':base_folder+'Pancreatic_data/Segerstolpe/Labels.csv',
                'labels_key':'x'},
    Xin={'data':base_folder+'Pancreatic_data/Xin/Filtered_Xin_HumanPancreas_data.csv',
                'labels':base_folder+'Pancreatic_data/Xin/Labels.csv',
            'labels_key':'x'},
    #TM
    TM={'data':base_folder+'TM/Filtered_TM_data.csv',
        'labels':base_folder+'TM/Labels.csv',
        'labels_key':'x'},
    #Zheng
    Zhengsorted={'data':base_folder+'Zheng sorted/Filtered_DownSampled_SortedPBMC_data.csv',
    'labels':base_folder+'Zheng sorted/Labels.csv',
    'labels_key':'x'},
    
    Zheng68k={'data':base_folder+'Zheng 68K/Filtered_68K_PBMC_data.csv',
    'labels':base_folder+'Zheng 68K/Labels.csv',
    'labels_key':'x'},
    
    #PbmcBench
    PbmcBench_10Xv2={'data':base_folder2+'PbmcBench/10Xv2/10Xv2_pbmc1.csv',
            'labels':base_folder2+'PbmcBench/10Xv2/10Xv2_pbmc1Labels.csv'},
    PbmcBench_10Xv3={'data':base_folder2+'PbmcBench/10Xv3/10Xv3_pbmc1.csv',
            'labels':base_folder2+'10Xv3/10Xv3_pbmc1Labels.csv'},
    PbmcBench_CELseq={'data':base_folder2+'PbmcBench/CEL-Seq/CL_pbmc1.csv',
            'labels':base_folder2+'PbmcBench/CEL-Seq/CL_pbmc1Labels.csv'},
    PbmcBench_Dropseq={'data':base_folder2+'PbmcBench/Drop-Seq/DR_pbmc1.csv',
            'labels':base_folder2+'PbmcBench/Drop-Seq/DR_pbmc1Labels.csv'},
    PbmcBench_inDrop={'data':base_folder2+'PbmcBench/inDrop/iD_pbmc1.csv',
            'labels':base_folder2+'PbmcBench/inDrop/iD_pbmc1Labels.csv'},
    PbmcBench_SeqWell={'data':base_folder2+'PbmcBench/Seq-Well/SW_pbmc1.csv',
            'labels':base_folder2+'PbmcBench/Seq-Well/SW_pbmc1Labels.csv'},
    
    PbmcBench_SmartSeq2={'data':base_folder2+'PbmcBench/Smart-Seq2/SM2_pbmc1.csv',
            'labels':base_folder2+'PbmcBench/Smart-Seq2/SM2_pbmc1Labels.csv'},
    
    #SCINA data : moignard, Paul (ignored for now)
    #VISP, ALM, MTG
)

# Run datasets

In [4]:
fnames=[i for i in all_data_paths.keys() if 'Pbmc' not in i and 'AMB' not in i and 'TM' not in i and 'Zheng' not in i]

In [5]:
#fixed params
do_log = True #already done with do_norm
do_pca = True
#n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))

In [5]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        adata = sc.read_csv(all_data_paths[fname]['data'])
        adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values

        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['labels'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['labels'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path(fname),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True duo True
cosine 25 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 500 True duo True
cosine 500 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 100 True seurat True
euclidean 100 True seurat False
euclidean 500 True duo True
euclidean 500 True duo False
euclidean 500 True seurat True
euclidean 500 True seurat False


In [6]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for fname in fnames:
        if get_res_path_sc(fname).split('/')[-1] in os.listdir(resdir):
            continue
        print('\t\t'+fname)
        adata = sc.read_csv(all_data_paths[fname]['data'])
        adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values

        results_dict = benchmark_common_functions.scanpy_hubness_analysis(
            adata,
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps
        )

        with open(get_res_path_sc(fname),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True duo True
		cellbench1
		duo norm retained 5000 genes
			Scoring: 0.25 mn
		cellbench2
		duo norm retained 5000 genes
			Scoring: 0.03 mn
		BaronHuman
		duo norm retained 5000 genes
			Scoring: 0.67 mn
		BaronMouse
		duo norm retained 5000 genes
			Scoring: 0.05 mn
		Muraro
		duo norm retained 5000 genes
			Scoring: 0.08 mn
		Segerstolpe
		duo norm retained 5000 genes
			Scoring: 0.11 mn
		Xin
		duo norm retained 5000 genes
			Scoring: 0.05 mn
cosine 25 True duo False
		cellbench1
		duo norm retained 5000 genes
			Scoring: 0.15 mn
		cellbench2
		duo norm retained 5000 genes
			Scoring: 0.02 mn
		BaronHuman
		duo norm retained 5000 genes
			Scoring: 0.44 mn
		BaronMouse
		duo norm retained 5000 genes
			Scoring: 0.07 mn
		Muraro
		duo norm retained 5000 genes
			Scoring: 0.08 mn
		Segerstolpe
		duo norm retained 5000 genes
			Scoring: 0.22 mn
		Xin
		duo norm retained 5000 genes
			Scoring: 0.06 mn
cosine 25 True seurat True
		cellbench1
		seurat norm retained 1795 genes
	

# Run TM dataset

In [3]:
#fixed params
do_log = True #already done with do_norm
do_pca = True
#n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))
# params_list = [i for i in params_list if not(i[-2]=='duo' and i[-1]==True)]

In [4]:
fname= 'TM'
f=csv.reader(open(all_data_paths[fname]['data'],'r'))
# n_dims=len(next(f))
# n_samples=[i for i,j in enumerate(f)]
n_samples, n_dims = (54865, 19791)

shape = (n_samples, n_dims)
mat = benchmark_common_functions.IncrementalCSRMatrix(shape, np.float32)

for row_ind,line in enumerate(f,start=-1):
    print(row_ind,end='\r')
    if row_ind==-1:
        continue
        
    data = np.array(line[1:],dtype=float)
    idx = data.nonzero()[0]
    for col_ind in idx:
        mat.append(row_ind, col_ind, data[col_ind])

In [ ]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path(fname).split('/')[-1] in os.listdir(resdir):
        continue
    adata = anndata.AnnData(X=mat.tocsr())
    adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values

    results_dict = benchmark_common_functions.clustering_analysis(
        adata,
        true_labels = adata.obs['labels'],
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        n_clusters= adata.obs['labels'].nunique(),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    with open(get_res_path(fname),'wb') as f:
        pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

cosine 25 True duo True
		duo norm retained 5000 genes
			Preprocessing done: 0.3 mn
			Inputs generated: 2.23 mn


In [5]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path_sc(fname).split('/')[-1] in os.listdir(resdir):
        continue
    adata = anndata.AnnData(X=mat.tocsr())
    adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values

    results_dict = benchmark_common_functions.scanpy_hubness_analysis(
        adata,
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    with open(get_res_path_sc(fname),'wb') as f:
        pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

cosine 25 True duo True
		duo norm retained 5000 genes


MemoryError: Unable to allocate 22.4 GiB for an array with shape (54865, 54865) and data type int64

# Run Zheng68k dataset

In [ ]:
#fixed params
do_log = True #already done with do_norm
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))
# params_list = [i for i in params_list if not(i[-2]=='duo' and i[-1]==True)]

In [ ]:
fname='Zheng68k'
f=csv.reader(open(all_data_paths[fname]['data'],'r'))
# n_dims=len(next(f))
# n_samples=[i for i,j in enumerate(f)]
n_samples,n_dims=(65943,20387)

shape = (n_samples, n_dims)
mat = benchmark_common_functions.IncrementalCSRMatrix(shape, np.float32)

for row_ind,line in enumerate(f,start=-1):
    print(row_ind,end='\r')
    if row_ind==-1:
        continue
        
    data = np.array(line[1:],dtype=float)
    idx = data.nonzero()[0]
    for col_ind in idx:
        mat.append(row_ind, col_ind, data[col_ind])

In [ ]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path(fname).split('/')[-1] in os.listdir(resdir):
        continue
    adata = anndata.AnnData(X=mat.tocsr())
    adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values
    sc.pp.subsample(adata,n_obs=20000)

    results_dict = benchmark_common_functions.clustering_analysis(
        adata,
        true_labels = adata.obs['labels'],
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        n_clusters= adata.obs['labels'].nunique(),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    with open(get_res_path(fname),'wb') as f:
        pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

cosine 25 True duo True
cosine 25 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 500 True duo True
cosine 500 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 100 True seurat True
		seurat norm retained 1756 genes
			Preprocessing done: 0.26 mn
			Inputs generated: 4.44 mn


In [ ]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path_sc(fname).split('/')[-1] in os.listdir(resdir):
        continue
    adata = anndata.AnnData(X=mat.tocsr())
    adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values
    sc.pp.subsample(adata,n_obs=20000)

    results_dict = benchmark_common_functions.scanpy_hubness_analysis(
        adata,
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    with open(get_res_path_sc(fname),'wb') as f:
        pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

# Run Zhengsorted dataset

In [3]:
#fixed params
do_log = True #already done with do_norm
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('seurat','duo')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))
# params_list = [i for i in params_list if not(i[-2]=='duo' and i[-1]==True)]

In [4]:
import scipy
fname='Zhengsorted'
f=csv.reader(open(all_data_paths[fname]['data'],'r'))
# n_dims=len(next(f))
# n_samples=[i for i,j in enumerate(f)]
n_samples, n_dims=(20000, 21952)

shape = (n_samples, n_dims)
mat = benchmark_common_functions.IncrementalCSRMatrix(shape, np.float32)

for row_ind,line in enumerate(f,start=-1):
    print(row_ind,end='\r')
    if row_ind==-1:
        continue
        
    data = np.array(line[1:],dtype=float)
    idx = data.nonzero()[0]
    for col_ind in idx:
        mat.append(row_ind, col_ind, data[col_ind])

In [5]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path(fname).split('/')[-1] in os.listdir(resdir):
        continue
    #adata = sc.read_csv(all_data_paths[fname]['data'])
    adata = anndata.AnnData(X=mat.tocsr())

    adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values

    results_dict = benchmark_common_functions.clustering_analysis(
        adata,
        true_labels = adata.obs['labels'],
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        n_clusters= adata.obs['labels'].nunique(),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    with open(get_res_path(fname),'wb') as f:
        pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

cosine 25 True seurat True
cosine 25 True seurat False
cosine 25 True duo True
cosine 25 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
cosine 500 True duo True
cosine 500 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 100 True seurat True
euclidean 100 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 500 True seurat True
euclidean 500 True seurat False
euclidean 500 True duo True
euclidean 500 True duo False
		duo norm retained 5000 genes
			Preprocessing done: 1.62 mn
			Inputs generated: 17.02 mn
			Scanpy lo

In [6]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path_sc(fname).split('/')[-1] in os.listdir(resdir):
        continue
    #adata = sc.read_csv(all_data_paths[fname]['data'])
    adata = anndata.AnnData(X=mat.tocsr())

    adata.obs['labels'] = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values

    results_dict = benchmark_common_functions.scanpy_hubness_analysis(
        adata,
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    with open(get_res_path_sc(fname),'wb') as f:
        pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

cosine 25 True seurat True
		seurat norm retained 2420 genes


			Scoring: 5.78 mn
cosine 25 True seurat False
		seurat norm retained 2420 genes


			Scoring: 3.92 mn
cosine 25 True duo True
		duo norm retained 5000 genes


			Scoring: 4.0 mn
cosine 25 True duo False
		duo norm retained 5000 genes


			Scoring: 4.16 mn
cosine 50 True seurat True
		seurat norm retained 2420 genes


...

			Scoring: 2.47 mn
euclidean 500 True seurat True
		seurat norm retained 2420 genes


			Scoring: 4.13 mn
euclidean 500 True seurat False
		seurat norm retained 2420 genes


			Scoring: 2.85 mn
euclidean 500 True duo True
		duo norm retained 5000 genes


			Scoring: 5.98 mn
euclidean 500 True duo False
		duo norm retained 5000 genes


			Scoring: 7.03 mn


# Run AMB dataset

In [7]:
#fixed params
do_log = True #already done with do_norm
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('seurat','duo')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))
# params_list = [i for i in params_list if not(i[-2]=='duo' and i[-1]==True)]

In [9]:
fname= 'AMB3'
f=csv.reader(open(all_data_paths[fname]['data'],'r'))
# n_samples=len([i for i,j in enumerate(f)])-1
# n_dims=len(next(f))-1
n_samples, n_dims= (12832,42625)

shape = (n_samples, n_dims)
mat = benchmark_common_functions.IncrementalCSRMatrix(shape, np.float32)

for row_ind,line in enumerate(f,start=-1):
    print(row_ind,end='\r')
    if row_ind==-1:
        continue
        
    data = np.array(line[1:],dtype=float)
    idx = data.nonzero()[0]
    for col_ind in idx:
        mat.append(row_ind, col_ind, data[col_ind])

In [5]:
for fname in ['AMB3','AMB16','AMB92']:
    for metric,n_comps,weighted,do_norm,norm_scale in params_list:
        print(metric,n_comps,weighted,do_norm,norm_scale)
        if get_res_path(fname).split('/')[-1] in os.listdir(resdir):
            continue
        labels = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values
        unique_labels, counts = np.unique(labels,return_counts=True)
        idx = np.where(~np.isin(labels,unique_labels[counts<=10]))[0]
        filtered_labels = labels[idx]
        adata = anndata.AnnData(X = mat.tocsr()[idx])
        adata.obs['labels'] = filtered_labels

        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            true_labels = adata.obs['labels'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors=int(round(np.sqrt(len(adata)))),
            n_clusters= adata.obs['labels'].nunique(),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps,
            retained_cells_idx=range(adata.X.shape[0])
        )

        with open(get_res_path(fname),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True seurat True
cosine 25 True seurat False
cosine 25 True duo True
cosine 25 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
cosine 500 True duo True
cosine 500 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 100 True seurat True
euclidean 100 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 500 True seurat True
euclidean 500 True seurat False
euclidean 500 True duo True
euclidean 500 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
cosine 25 True duo True
cosine 25 True duo False


In [10]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    if get_res_path_sc(fname).split('/')[-1] in os.listdir(resdir):
        continue
    labels = pd.read_csv(all_data_paths[fname]['labels'])[all_data_paths[fname]['labels_key']].values
    unique_labels, counts = np.unique(labels,return_counts=True)
    idx = np.where(~np.isin(labels,unique_labels[counts<=10]))[0]
    filtered_labels = labels[idx]
    adata = anndata.AnnData(X = mat.tocsr()[idx])
    adata.obs['labels'] = filtered_labels

    results_dict = benchmark_common_functions.scanpy_hubness_analysis(
        adata,
        do_norm = do_norm,
        norm_scale = norm_scale,
        do_log = do_log,
        do_pca = do_pca,
        n_neighbors=int(round(np.sqrt(len(adata)))),
        metric=metric,
        weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
        seed=seed,
        n_comps=n_comps,
        retained_cells_idx=range(adata.X.shape[0])
    )

    for fname in ['AMB3','AMB16','AMB92']:
        with open(get_res_path_sc(fname),'wb') as f:
            pickle.dump(results_dict,f)
    %xdel adata
    gc.collect()

cosine 25 True duo True
cosine 25 True duo False
cosine 25 True seurat True
cosine 25 True seurat False
cosine 50 True duo True
cosine 50 True duo False
cosine 50 True seurat True
cosine 50 True seurat False
cosine 100 True duo True
cosine 100 True duo False
cosine 100 True seurat True
cosine 100 True seurat False
cosine 500 True duo True
cosine 500 True duo False
cosine 500 True seurat True
cosine 500 True seurat False
euclidean 25 True duo True
euclidean 25 True duo False
euclidean 25 True seurat True
euclidean 25 True seurat False
euclidean 50 True duo True
euclidean 50 True duo False
euclidean 50 True seurat True
euclidean 50 True seurat False
euclidean 100 True duo True
euclidean 100 True duo False
euclidean 100 True seurat True
euclidean 100 True seurat False
euclidean 500 True duo True
		duo norm retained 5000 genes


			Scoring: 2.55 mn
euclidean 500 True duo False
		duo norm retained 5000 genes


			Scoring: 2.06 mn
euclidean 500 True seurat True
euclidean 500 True seurat False
